### Importing Data and Required Packages

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# modelling 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [19]:
df = pd.read_csv(r"C:\Users\kanis\Desktop\Breast-Cancer-Prediction\notebook\raw.csv")

In [ ]:
df.head()

In [21]:
df.drop("id",axis = 1 ,inplace=True)

In [ ]:
df = df.replace({"M":1,"B":0})

### Seperating out Features and label

In [23]:
y = df["diagnosis"]
x = df.drop("diagnosis",axis=1)

### Train-Test Split

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

### Feature Scaling

In [35]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

### Applying GridSearchCV

In [45]:
models = {
    "LogisticRegression":{
        "model_name":LogisticRegression(),
        "parameters":{
          "C":[0.1,1,10],
          "penalty":["l1","l2"],
          "solver":["liblinear","saga"]
    }
},
    "RandomForest":{
        "model_name":RandomForestClassifier(random_state = 42),
        "parameters":{
          "n_estimators": [50, 100, 200],
          "max_depth": [None, 10, 20],
          "min_samples_split": [2, 5, 10]
    }
},  
    "svm":{
        "model_name":SVC(),
        "parameters":{
          "C":[0.01,0.1,1],
          "gamma":[0.01,0.001,1,10],
          "kernel":["rbf","linear"]
    }
}
    }

In [46]:
best_models = {}
for i in models:
    clf= GridSearchCV(
        models[i]["model_name"],
        models[i]["parameters"],cv=5,n_jobs=-1)
    clf.fit(x_train,y_train)
    best_models[i] = clf.best_estimator_
    print(f"Best parameters for {i} : {clf.best_params_}")


Best parameters for LogisticRegression : {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best parameters for RandomForest : {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best parameters for svm : {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}


### Finding out Metrics for each Model

In [48]:
for model_name,model in best_models.items():
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("\n"f"{model_name} performance on Test set:")
    print(f"Accuracy: {accuracy}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")



LogisticRegression performance on Test set:
Accuracy: 0.9824561403508771
precision: 0.9829367940398942
recall: 0.9824561403508771
f1: 0.9823691172375383

RandomForest performance on Test set:
Accuracy: 0.9649122807017544
precision: 0.9652053622194477
recall: 0.9649122807017544
f1: 0.9647382344750765

svm performance on Test set:
Accuracy: 0.9824561403508771
precision: 0.9829367940398942
recall: 0.9824561403508771
f1: 0.9823691172375383


### Applying KFold Cross-Validation

In [55]:
for model_name,model in best_models.items():
    kf = KFold(n_splits=5, shuffle = True , random_state =42)
    scores = cross_val_score(model,x,y,cv=kf)
    print("\n"f"Scores for {model_name}: {scores}")
    print(f"Average score: {np.average(scores)}")


Scores for LogisticRegression: [0.96491228 0.99122807 0.89473684 0.94736842 0.92920354]
Average score: 0.9454898307716194

Scores for RandomForest: [0.96491228 0.97368421 0.94736842 0.96491228 0.96460177]
Average score: 0.9630957925787922

Scores for svm: [0.96491228 0.95614035 0.94736842 0.96491228 0.9380531 ]
Average score: 0.9542772861356932


From the above analysis it can be deduced that the model which is giving consistent scores and highest average of scores on cross-validation is RandomForestClassifier